In [1]:
from tensorflow.keras import layers, models, Model, Input
import pandas as pd
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np
from numpy import asarray
from sklearn import model_selection, metrics
import seaborn as sns
from sklearn.model_selection import ShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping
import random
labels = pd.read_csv('codes_categorized.csv', index_col = 'id')
column_name = "communications_facilities"
model_folder = "results/updated_models_removed_correlated/" + column_name + "/"
len(labels)

64

In [2]:
def loadCategorical():
    block_folder = "block_data/preprocessed/"
    categorical = pd.read_csv(block_folder + 'block_counts_normalized_removed_correlated.csv', index_col = 'id')
    return categorical

categorical = loadCategorical()
len(categorical)

57

In [3]:
for index, row in labels.iterrows():
    if index not in categorical.index:
        labels = labels.drop([index])
len(labels)

57

In [4]:
labels

,oxygen_regulation,radiation_protection,gravity,food,supplies,power_generation,health,communications_facilities,shape,area,transportation
id,,,,,,,,,,,
1,2.0,1.0,0,2.0,0.0,2.0,1,2.0,1.0,0.0,0.0
2,2.0,0.0,0,0.0,0.0,2.0,0,2.0,2.0,2.0,0.0
3,2.0,2.0,1,2.0,2.0,2.0,0,2.0,1.0,2.0,2.0
4,2.0,2.0,1,1.0,2.0,2.0,0,2.0,2.0,0.0,0.0
5,2.0,2.0,1,2.0,0.0,1.0,0,2.0,0.0,1.0,2.0
6,2.0,2.0,1,2.0,2.0,2.0,0,2.0,2.0,2.0,0.0
7,2.0,0.0,1,0.0,2.0,2.0,1,1.0,1.0,2.0,0.0
8,2.0,2.0,0,1.0,0.0,2.0,0,2.0,2.0,0.0,0.0
9,2.0,0.0,1,0.0,0.0,2.0,0,0.0,0.0,2.0,0.0


In [5]:
def generateNumLayers(num_options):
    return random.randint(1, num_options)
def generateNumNodes(num_layers, start, end):
    num_nodes_list = []
    for i in range(0, num_layers):
        num_nodes = random.randint(start, end)
        num_nodes_list.append(num_nodes)
    return num_nodes_list
def createModelBlockData(num_classes, num_layers, num_nodes):
    inputB = Input(shape=(len(categorical.columns)))
    for i in range(0, num_layers):
        if i == 0:
            x = layers.Dense(num_nodes[i], activation='relu')(inputB)
        else:
            x = layers.Dense(num_nodes[i], activation='relu')(x)
        x = layers.Dropout(.4)(x)
    x = layers.Dense(num_classes, activation = 'softmax')(x)
    x = Model(inputs=inputB, outputs=x)
    x.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
    return x

In [6]:
splits = 5
k_fold = model_selection.StratifiedKFold(n_splits= splits, shuffle = True, random_state = 42)
epochs=1000
iterations = 50
results = {
    }  
for x in range(0, iterations):
    train_labels = labels[column_name].to_numpy()
    class_labels = np.unique(train_labels)
    num_classes = len(class_labels)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=train_labels)
    weight_dict = dict(zip(class_labels, class_weights))
    num_layers = generateNumLayers(4)
    num_nodes = generateNumNodes(num_layers, 25, 500)
    accuracy = []
    precision = []
    recall = []
    f1 = []
    print(x)
    for i, (train_indices, test_indices) in enumerate(k_fold.split(categorical, train_labels)):
        print("Split " + str(i+1))
        X_train = []
        y_train = []
        X_test = []
        y_test = []
        for index in train_indices:
            X_train.append(categorical.iloc[[index]].values.tolist()[0])
            y_train.append(train_labels[index])
        for index in test_indices:
            X_test.append(categorical.iloc[[index]].values.tolist()[0])
            y_test.append(train_labels[index])
        es = EarlyStopping(monitor='loss', mode='min', patience=3, start_from_epoch = 50,  restore_best_weights=True)
        # models
        model_blocks = createModelBlockData(num_classes, num_layers, num_nodes)
        X_train = np.array(X_train).astype(np.float32)
        history = model_blocks.fit(
        [X_train],
        np.array(y_train),
        class_weight = weight_dict,
        epochs=epochs,
        callbacks=[es]
        )
        X_test = np.array(X_test).astype(np.float32)
        predictions = model_blocks.predict([X_test])
        y_pred = []
        for k in range(len(predictions)):
            y_pred.append(np.argmax(predictions[k]))
        #Define scoring metrics
        accuracy.append(metrics.accuracy_score(y_test, y_pred))
        precision.append(metrics.precision_score(y_test, y_pred, average = 'macro', zero_division = np.nan))
        recall.append(metrics.recall_score(y_test, y_pred, average = 'macro', zero_division = np.nan))
        f1.append(metrics.f1_score(y_test, y_pred, average = 'macro', zero_division = np.nan)) 
    results[x] = {'num_layers': num_layers,
                          'num_nodes': num_nodes,
                          'accuracy': np.average(accuracy),
    'precision': np.average(precision),
    'recall': np.average(recall),
    'f1_score': np.average(f1),
        }

0
Split 1
Epoch 1/1000


C:\Users\humdi\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


2/2 [==============================] - 0s 11ms/step - loss: 1.5905 - accuracy: 0.4667
Epoch 2/1000
2/2 [==============================] - 0s 4ms/step - loss: 1.4069 - accuracy: 0.5778
Epoch 3/1000
2/2 [==============================] - 0s 4ms/step - loss: 1.7103 - accuracy: 0.5778
Epoch 4/1000
2/2 [==============================] - 0s 3ms/step - loss: 1.0279 - accuracy: 0.6000
Epoch 5/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.7787 - accuracy: 0.6667
Epoch 6/1000
2/2 [==============================] - 0s 4ms/step - loss: 1.0517 - accuracy: 0.6444
Epoch 7/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.6156 - accuracy: 0.7111
Epoch 8/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.3774 - accuracy: 0.8222
Epoch 9/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.4042 - accuracy: 0.7333
Epoch 10/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.3604 - accuracy: 0.8222
Epoch 11/1000
2/2 [============

2/2 [==============================] - 0s 5ms/step - loss: 0.0924 - accuracy: 0.9778
Epoch 27/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.1283 - accuracy: 0.9556
Epoch 28/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.0777 - accuracy: 0.9556
Epoch 29/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.2112 - accuracy: 0.9778
Epoch 30/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.0978 - accuracy: 0.9778
Epoch 31/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.0942 - accuracy: 0.9556
Epoch 32/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.0395 - accuracy: 1.0000
Epoch 33/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.0627 - accuracy: 0.9778
Epoch 34/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.2773 - accuracy: 0.9778
Epoch 35/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.0938 - accuracy: 1.0000
Epoch 36/1000
2/2 [=====

1/1 [==============================] - 0s 50ms/step
Split 4
Epoch 1/1000
2/2 [==============================] - 0s 4ms/step - loss: 1.4060 - accuracy: 0.2391
Epoch 2/1000
2/2 [==============================] - 0s 5ms/step - loss: 1.1503 - accuracy: 0.3261
Epoch 3/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.7812 - accuracy: 0.4565
Epoch 4/1000
2/2 [==============================] - 0s 3ms/step - loss: 0.5633 - accuracy: 0.5217
Epoch 5/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.6873 - accuracy: 0.4565
Epoch 6/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.3854 - accuracy: 0.8043
Epoch 7/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.3383 - accuracy: 0.7609
Epoch 8/1000
2/2 [==============================] - 0s 4ms/step - loss: 0.4809 - accuracy: 0.7609
Epoch 9/1000
2/2 [==============================] - 0s 5ms/step - loss: 0.4391 - accuracy: 0.8261
Epoch 10/1000
2/2 [==============================] - 0s 8m

...

C:\Users\humdi\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


2/2 [==============================] - 1s 14ms/step - loss: 1.7564 - accuracy: 0.6444
Epoch 2/1000
2/2 [==============================] - 0s 7ms/step - loss: 1.8646 - accuracy: 0.5778
Epoch 3/1000
2/2 [==============================] - 0s 7ms/step - loss: 1.7521 - accuracy: 0.6444
Epoch 4/1000
2/2 [==============================] - 0s 7ms/step - loss: 2.0072 - accuracy: 0.6889
Epoch 5/1000
2/2 [==============================] - 0s 7ms/step - loss: 0.8399 - accuracy: 0.6000
Epoch 6/1000
2/2 [==============================] - 0s 8ms/step - loss: 1.2653 - accuracy: 0.7556
Epoch 7/1000
2/2 [==============================] - 0s 7ms/step - loss: 0.6497 - accuracy: 0.8222
Epoch 8/1000
2/2 [==============================] - 0s 7ms/step - loss: 0.3959 - accuracy: 0.7556
Epoch 9/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.3845 - accuracy: 0.8444
Epoch 10/1000
2/2 [==============================] - 0s 8ms/step - loss: 0.4409 - accuracy: 0.9333
Epoch 11/1000
2/2 [============

2/2 [==============================] - 0s 7ms/step - loss: 0.0228 - accuracy: 0.9778
Epoch 30/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.0215 - accuracy: 0.9778
Epoch 31/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.0232 - accuracy: 0.9778
Epoch 32/1000
2/2 [==============================] - 0s 7ms/step - loss: 0.0176 - accuracy: 0.9778
Epoch 33/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.0131 - accuracy: 1.0000
Epoch 34/1000
2/2 [==============================] - 0s 7ms/step - loss: 0.0178 - accuracy: 1.0000
Epoch 35/1000
2/2 [==============================] - 0s 7ms/step - loss: 0.0071 - accuracy: 1.0000
Epoch 36/1000
2/2 [==============================] - 0s 8ms/step - loss: 0.0085 - accuracy: 1.0000
Epoch 37/1000
2/2 [==============================] - 0s 8ms/step - loss: 0.0054 - accuracy: 1.0000
Epoch 38/1000
2/2 [==============================] - 0s 7ms/step - loss: 0.0673 - accuracy: 0.9778
Epoch 39/1000
2/2 [=====

2/2 [==============================] - 0s 8ms/step - loss: 6.3524e-04 - accuracy: 1.0000
Epoch 54/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 55/1000
2/2 [==============================] - 0s 6ms/step - loss: 8.2819e-04 - accuracy: 1.0000
Epoch 56/1000
2/2 [==============================] - 0s 7ms/step - loss: 3.4135e-04 - accuracy: 1.0000
Epoch 57/1000
2/2 [==============================] - 0s 6ms/step - loss: 5.1575e-04 - accuracy: 1.0000
Epoch 58/1000
2/2 [==============================] - 0s 8ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 59/1000
1/1 [==============================] - 0s 62ms/step
Split 4
Epoch 1/1000
2/2 [==============================] - 1s 4ms/step - loss: 1.2309 - accuracy: 0.3478
Epoch 2/1000
2/2 [==============================] - 0s 7ms/step - loss: 0.8980 - accuracy: 0.6304
Epoch 3/1000
2/2 [==============================] - 0s 7ms/step - loss: 1.0250 - accuracy: 0.6087
Epoch 4/1000
2/2 [==================

2/2 [==============================] - 0s 6ms/step - loss: 0.0479 - accuracy: 0.9783
Epoch 21/1000
2/2 [==============================] - 0s 7ms/step - loss: 0.0517 - accuracy: 0.9783
Epoch 22/1000
2/2 [==============================] - 0s 7ms/step - loss: 0.0421 - accuracy: 0.9783
Epoch 23/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.0158 - accuracy: 1.0000
Epoch 24/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.1307 - accuracy: 1.0000
Epoch 25/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.0218 - accuracy: 1.0000
Epoch 26/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.0383 - accuracy: 1.0000
Epoch 27/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.0272 - accuracy: 0.9783
Epoch 28/1000
2/2 [==============================] - 0s 7ms/step - loss: 0.0137 - accuracy: 1.0000
Epoch 29/1000
2/2 [==============================] - 0s 6ms/step - loss: 0.0122 - accuracy: 1.0000
Epoch 30/1000
2/2 [=====

In [7]:
results_df = pd.DataFrame(results)
results_df.to_csv(model_folder + "results_cleaned.csv")

In [8]:
best = 0
best_index = -1
best_tied = []
row_name = "accuracy"
for index, row in results_df.loc[row_name].iteritems():
    value = row
    if value > best:
        best = value
        best_tied = [index]
        best_index = index
    elif value == best:
        best_tied.append(index)
print("Accuracy:")
print(best_index)
print(best_tied)
print(best)

Accuracy:
2
[2]
0.6833333333333332


In [9]:
if len(best_tied) == 1:
    print("Layers: " + str(results_df[best_index][0]))
    print("Nodes: " + str(results_df[best_index][1]))
else:
    index = -1
    best_avg = -1
    for i in range(len(best_tied)):
        element = results_df[best_tied[i]]
        avg = 0
        for j in range(2, len(element), 1): 
            avg += element[j]
        if avg > best_avg:
            index = best_tied[i]
            best_avg = avg
    print("Layers: " + str(results_df[index][0]))
    print("Nodes: " + str(results_df[index][1]))

Layers: 4
Nodes: [397, 311, 324, 329]


In [10]:
best = 0
best_index = -1
best_tied = []
row_name = "precision"
for index, row in results_df.loc[row_name].iteritems():
    value = row
    if value > best:
        best = value
        best_tied = [index]
        best_index = index
    elif value == best:
        best_tied.append(index)
print("Precision:")
print(best_index)
print(best_tied)
print(best)

Precision:
2
[2]
0.6791558441558443


In [11]:
best = 0
best_index = -1
best_tied = []
row_name = "recall"
for index, row in results_df.loc[row_name].iteritems():
    value = row
    if value > best:
        best = value
        best_tied = [index]
        best_index = index
    elif value == best:
        best_tied.append(index)
print("Recall:")
print(best_index)
print(best_tied)
print(best)

Recall:
45
[45]
0.5978174603174603


In [12]:
best = 0
best_index = -1
best_tied = []
row_name = "f1_score"
for index, row in results_df.loc[row_name].iteritems():
    value = row
    if value > best:
        best = value
        best_tied = [index]
        best_index = index
    elif value == best:
        best_tied.append(index)
print("F1:")
print(best_index)
print(best_tied)
print(best)

F1:
45
[45]
0.6617668605903899
